In [1]:
import circle_cal.model as ccm
from workalendar.usa import UnitedStates, Indiana
from workalendar.core import ChristianMixin
from datetime import datetime, date, time
from datetime import timedelta
import calendar

In [2]:
y24 = ccm.Year(2024)

In [3]:
y24.THETA_PER_DAY

0.9836065573770492

In [4]:
for m in y24:
    print(m.name)

January
February
March
April
May
June
July
August
September
October
November
December


In [10]:
m.start, m.end

(datetime.datetime(2024, 12, 1, 0, 0), datetime.datetime(2024, 12, 31, 0, 0))

In [6]:
d = date(2024, 1, 1)

In [7]:
d

datetime.date(2024, 1, 1)

In [8]:
d + timedelta(days=31)

datetime.date(2024, 2, 1)

In [9]:
y24.len_by_days()

366.0

In [13]:
calendar.month_name[1]

'January'